In [3]:
import glob
import os
from lsst import afw
print(afw.version.__version__)
from lsst.daf.persistence import Butler
from lsst.eotest.sensor import MaskedCCD, makeAmplifierGeometry
import numpy

from exploreRun import exploreRun
from lsst.eo_utils.base.image_utils import get_ccd_from_id,\
    get_amp_list, get_data_as_read, sort_sflats

from lsst.eo_utils.base.data_access import get_data_for_run

import lsst.eotest.image_utils as imutil
import pylab

from astropy.io import fits
from astropy.stats import mad_std
import numpy as np

import scipy
from scipy import signal
from scipy.fftpack import fft, fftshift
import matplotlib.pyplot as plt

import math
from scipy.ndimage import gaussian_filter

#from mytool import

%matplotlib inline

20.0.0-4-gde602ef96+1


In [4]:
superbiaspath = "/gpfs/slac/lsst/fs1/u/echarles/DATA/analysis/ts8/superbias/RTM-013/RTM-013-10982-S21_superbias_normal.fits"
superdarkpath = "/gpfs/slac/lsst/fs1/u/echarles/DATA/analysis/ts8/superdark/RTM-013/RTM-013-10982-S21_superdark_normal.fits"
qeflatpath = glob.glob("/gpfs/slac/lsst/fs3/g/data/jobHarness/jh_stage/LCA-11021_RTM/LCA-11021_RTM-013/10982/qe_raft_acq/v0/85192/S21/*_lambda_flat_*")
qeflatpath.sort()

# array of wavelengths
wls = numpy.array([ get_ccd_from_id(None, aqeflatpath, []).md.md["MONOWL"] for aqeflatpath in qeflatpath ])

wavelengths_list = [365, 466.6, 622.6, 759.9, 845.7, 950.5]
width_of_wavelength_list = [12, 25, 18, 28, 32, 60]
width_of_wavelength_list_converted = []
for num in width_of_wavelength_list:
    z = num / 2.355
    width_of_wavelength_list_converted.append(z)

mu = wavelengths_list[0]     # UV band
sigma = width_of_wavelength_list_converted[0]

a = 1    # to normalize
x = numpy.arange(300,1100,1)  
y = a*numpy.exp(-(x-mu)**2/(2*sigma**2))
# plots the gaussians
weights = numpy.interp(wls, x, y)   

In [ ]:
# loading all 5 position files
fname1= "/gpfs/slac/lsst/fs1/u/combet/DATA/CCOB_QE/fits/QE_R02_S21_red_-201.815_-10.575.fits"
fname2= "/gpfs/slac/lsst/fs1/u/combet/DATA/CCOB_QE/fits/QE_R02_S21_red_-201.815_9.785.fits"
fname3= "/gpfs/slac/lsst/fs1/u/combet/DATA/CCOB_QE/fits/QE_R02_S21_red_-211.815_-0.395.fits"
fname4= "/gpfs/slac/lsst/fs1/u/combet/DATA/CCOB_QE/fits/QE_R02_S21_red_-221.815_-10.575.fits"
fname5= "/gpfs/slac/lsst/fs1/u/combet/DATA/CCOB_QE/fits/QE_R02_S21_red_-221.815_9.785.fits"
file_list = [fname1, fname2, fname3, fname4, fname5]



def create_ccob(img_list):
    num_ch = 16
    uv_arr = [None] * num_ch
    normedimg_arr = [None] * num_ch
    MEDIAN = None

    fname = "/gpfs/slac/lsst/fs1/u/combet/DATA/CCOB_QE/fits/QE_R02_S21_uv_-211.815_-0.395.fits"     
    ccob_uv = get_ccd_from_id(None, fname, [])

    # reading in files in CCOB testing at uv LED for each of 16 channels 
    for ch in range(num_ch):    
        uv = get_data_as_read(ccob_uv,ch + 1,["imaging"])["imaging"]    # QE data array for CCOBuv flat field image
        MEDIAN = numpy.median(uv)                                        # median (not mean, avoiding outliers) of QE data array for CCOBuv, will normalize COMBINED with this
        normedimg = img_list[ch] / numpy.median(img_list[ch]) * MEDIAN    # normalizing COMBINED against the CCOB amp-by-amp to the level of the CCOB median 
        normedimg_arr[ch] = normedimg                                     # adding data at this amp to data array
        uv_arr[ch] = uv
    return normedimg_arr, uv_arr

In [ ]:
title = "COMBINED not normalized R02_S21"
img_list = create_combined_ITL(superdarkpath, superbiaspath, qeflatpath, weights, title)    # create_combined returns img_list at end of function
create_ccob(img_list)
print('')

In [ ]:
# default color map
num_ch = 16
normedimg_arr, uv_arr = create_ccob(img_list)

# plot normalized, COMBINED flat
plotonesensor_ITL(normedimg_arr, 'COMBINED normalized against CCOB$_{UV}$ R02_S21')

# plot CCOBred flat
plotonesensor_ITL(uv_arr, 'CCOB$_{UV}$ R02_S21')

# plot Ratio flat, the one of interest!
diff_arr = [None] * num_ch
for i in range(num_ch):
    diff_arr[i] = (normedimg_arr[i] - uv_arr[i]) / uv_arr[i]
plotonesensor_ITL(diff_arr, 'Differential Ratio')

In [39]:
### testing averaging 5 CCOB head positions
# fix, make cleaner into for loop

red_top = []
red_bot = []
normedimg_top = []
normedimg_bot = []

mean = None
num_ch  = 16
columns = 8
rows = 2

# loading all 5 position files
fname1= "/gpfs/slac/lsst/fs1/u/combet/DATA/CCOB_QE/fits/QE_R02_S21_red_-201.815_-10.575.fits"
fname2= "/gpfs/slac/lsst/fs1/u/combet/DATA/CCOB_QE/fits/QE_R02_S21_red_-201.815_9.785.fits"
fname3= "/gpfs/slac/lsst/fs1/u/combet/DATA/CCOB_QE/fits/QE_R02_S21_red_-211.815_-0.395.fits"
fname4= "/gpfs/slac/lsst/fs1/u/combet/DATA/CCOB_QE/fits/QE_R02_S21_red_-221.815_-10.575.fits"
fname5= "/gpfs/slac/lsst/fs1/u/combet/DATA/CCOB_QE/fits/QE_R02_S21_red_-221.815_9.785.fits"
file_list = [fname1, fname2, fname3, fname4, fname5]

# creating a list of the metadata arrays for each fname
ccob_list = []
for i in range(len(file_list)):
    ccob_red = get_ccd_from_id(None, file_list[i], [])
    ccob_list.append(ccob_red)

# looping over each amplifier in CCD
for ch in range(num_ch):
    
    # creating a list of the QE data arrays (at this amplifier) for each file
    ccob_position_list = []
    for i in range(5):
        red = get_data_as_read(ccob_list[i],ch + 1,["imaging"])["imaging"]   # use the list with metadata arrays here (ccob_list)
        ccob_position_list.append(red)
        
    # calculating the mean QE data array (at this amplifier) from all 5 files   
    ccob_sum = 0
    for ccob in ccob_position_list:
        ccob_sum += ccob
    red_avg = ccob_sum / 5 
    
    if ch < columns:
        # calculating the mean of the QE data array (at this amplifier/channel) for the top row of amplifiers
        mean_sum = 0
        for i in range(5):
            mean_sum += get_data_as_read(ccob_list[i],columns-ch,["imaging"])["imaging"]
        mean = mean_sum.mean()  # used to normalize the amplifiers in the top row
        normedimg = img_list_top_row[ch]/img_list_top_row[ch].mean()*mean
        normedimg_top.append(normedimg)
        red_top.append(red_avg)
        
    else:
        mean = red_avg  # used to normalize the amplifiers in the bottom row
        normedimg = img_list_bottom_row[ch - columns]/img_list_bottom_row[ch - columns].mean()*mean
        normedimg_bot.append(normedimg)
        red_bot.append(red_avg)

red_top.reverse()
print(ccob_position_list)
print()
print(ccob_sum)
print()
print(red_avg)
#print(red_top)
#print(red_top[2])

[array([[51610., 49785., 48679., ..., 44870., 44187., 44562.],
       [55745., 55247., 54886., ..., 51679., 50958., 51026.],
       [58779., 57974., 56711., ..., 55795., 53934., 53496.],
       ...,
       [67410., 66149., 66529., ..., 64369., 63900., 62846.],
       [66050., 65250., 66517., ..., 64630., 64696., 62980.],
       [66178., 66039., 67037., ..., 64956., 63613., 62684.]],
      dtype=float32), array([[51559., 49889., 48530., ..., 44478., 44336., 44785.],
       [55839., 55519., 55108., ..., 51507., 50141., 50987.],
       [58470., 57854., 56946., ..., 55368., 53438., 53525.],
       ...,
       [66911., 65600., 65642., ..., 63723., 64154., 62790.],
       [66128., 65221., 66616., ..., 63994., 64374., 62426.],
       [65935., 66099., 66312., ..., 64324., 63709., 62659.]],
      dtype=float32), array([[51250., 50115., 48248., ..., 43758., 43874., 44867.],
       [55516., 55144., 55351., ..., 51205., 49960., 50883.],
       [58843., 57685., 57289., ..., 54781., 53319., 53119.],